In [1]:
import os
import sys
import numpy as np
import cv2
import yaml

In [2]:
# 0:
# - cam_R_m2c: [0.09506610, 0.98330897, -0.15512900, 0.74159598, -0.17391300, -0.64791101, -0.66407597, -0.05344890, -0.74575198]
#   cam_t_m2c: [71.62781422, -158.20064191, 1050.77777823]
#   obj_bb: [331, 130, 65, 64]
#   obj_id: 6

In [3]:
data_path = r"/home/chris/Data/Projects/Pose Estimation/data/cube_2_sides"
image_path = os.path.join(data_path, "images")
label_path = os.path.join(data_path, "labels")

output_path = r"/home/chris/Data/Projects/Pose Estimation/EfficientPose/data/xray/cube/01"


In [4]:
def find(str, ch):
    for i, ltr in enumerate(str):
        if ltr == ch:
            yield i

In [5]:
def convert_normalized_to_pixel_coordinates(x, y, width, height):
    return int(x * width), int(y * height)

In [6]:
def convert_center_bb_to_corner_bb(x, y, width, height):
    return int(x - width / 2), int(y - height / 2), width, height


In [7]:
def calculate_focal_length(system_focal_length, image_with=None, image_height=None, detector_width=None, detector_height=None):
    focal_x = image_with/detector_width*system_focal_length
    focal_y = image_height/detector_height*system_focal_length
    return focal_x, focal_y

In [8]:
def get_camera_intrinsic(u0, v0, fx, fy):
    return np.array([[fx, 0.0, u0], [0.0, fy, v0], [0.0, 0.0, 1.0]])

In [9]:
def extract_info(line):

    first_part = line[:line.index("[[")]
    second_part = line[line.index("[["):]
    first_part.split(" ")
    print(second_part)
    rotation_string = second_part[second_part.index("[[")+2:second_part.index("]]")]
    print(rotation_string.split(" "))
    temp_list = []
    for t_item in rotation_string.split(" "):
        if t_item != "":
            temp_list.append(t_item)
    roation_list = []
    roation_list.append(temp_list[0])
    roation_list.append(temp_list[1])
    roation_list.append(temp_list[2])

    cam_R_m2c = cv2.Rodrigues(np.array(roation_list, dtype=np.float32))[0].flatten()
    print(cam_R_m2c)

    print(list(find(second_part, "]")))
    print(second_part)
    trans_string = second_part[second_part.index(" [")+2:  list(find(second_part, "]"))[2]]
    print(trans_string)
    temp_list = []
    for t_item in trans_string.split(" "):
        if t_item != "":
            temp_list.append(t_item)

    trans_list = []
    trans_list.append(temp_list[0])
    trans_list.append(temp_list[1])
    trans_list.append(temp_list[2])

    cam_t_m2c = np.array(trans_list, dtype=np.float32)
    
    coords_list = first_part.split(" ")
    print(coords_list)
    print(coords_list[1:3])
    print(coords_list[1+2*9:3+2*9])

    center_x = float(coords_list[1])
    center_y = float(coords_list[2])

    width = float(coords_list[1+2*9])
    height = float(coords_list[2+2*9])
    print(f"object width {width}, object height {height}")

    

    focal_length = float(coords_list[3+2*9])
    print(focal_length)
    detector_width = float(coords_list[4+2*9])
    print(detector_width)
    detector_height = float(coords_list[5+2*9])
    print(detector_height)
    img_center_x = float(coords_list[6+2*9])
    print(img_center_x)
    img_center_y = float(coords_list[7+2*9])
    print(img_center_y)
    img_width = float(coords_list[8+2*9])
    print(img_width)
    img_height = float(coords_list[9+2*9])
    print(img_height)

    focal_x, focal_y = calculate_focal_length(focal_length, image_with=img_width, image_height=img_height, detector_width=detector_width, detector_height=detector_height)
    fx                  = focal_x
    fy                  = -focal_y

    cam_matrix = get_camera_intrinsic(img_center_x, img_center_y, fx, fy).flatten()
    obj_id = 0

    center_x, center_y = convert_normalized_to_pixel_coordinates(center_x, center_y, img_width, img_height)
    print(center_x, center_y)
    
    width, height = convert_normalized_to_pixel_coordinates(width, height, img_width, img_height)
    print(width, height)

    obj_bb = convert_center_bb_to_corner_bb(center_x, center_y, width, height)
    print(obj_bb)

    return cam_R_m2c, cam_t_m2c, cam_matrix, list(obj_bb), obj_id

In [10]:
gt_dict = {}
info_dict = {}

In [11]:
for items in os.listdir(label_path):
    print(items)
    with open(os.path.join(label_path, items)) as f:
        lines = f.readlines()
        print(extract_info(lines[0]))
        ext_info = extract_info(lines[0])
        gt_dict[items[:-4]] = {'cam_R_m2c': ext_info[0].tolist(), 'cam_t_m2c': ext_info[1].tolist(), 'obj_bb': ext_info[3], 'obj_id': ext_info[4], 'cam_K': ext_info[2].tolist()}
        info_dict[items[:-4]] = {'cam_K': ext_info[2].tolist()}

Screenshot 2022-05-26 15_41_58_226306.txt
[[ 2.22535084  2.21226567 -0.02349527]] [ 0.00387468 -0.00126375  0.72971475]
['', '2.22535084', '', '2.21226567', '-0.02349527']
[ 0.00584418  0.9999504  -0.00806595  0.99989617 -0.00594975 -0.01312599
 -0.01317333 -0.0079884  -0.9998813 ]
[37, 38, 76]
[[ 2.22535084  2.21226567 -0.02349527]] [ 0.00387468 -0.00126375  0.72971475]
 0.00387468 -0.00126375  0.72971475
['0', '0.516347348690033', '0.5067536234855652', '0.4541677236557007', '0.5836224555969238', '0.4512031376361847', '0.5892781615257263', '0.5803930759429932', '0.5902818441390991', '0.578155517578125', '0.5845839381217957', '0.5811930894851685', '0.4265888035297394', '0.5789220929145813', '0.42748409509658813', '0.4519314467906952', '0.42563945055007935', '0.45486825704574585', '0.42657268047332764', '0.12998995', '0.1646424', '1.1699', '0.3799999952316284', '0.30000001192092896', '480.0', '371.0', '960', '742', '']
['0.516347348690033', '0.5067536234855652']
['0.12998995', '0.164642

In [12]:
len(gt_dict.keys())

681

In [13]:
gt_dict['Screenshot 2022-05-26 15_41_58_226306']

{'cam_R_m2c': [0.005844184663146734,
  0.9999504089355469,
  -0.008065949194133282,
  0.9998961687088013,
  -0.005949746817350388,
  -0.013125993311405182,
  -0.013173332437872887,
  -0.007988400757312775,
  -0.9998813271522522],
 'cam_t_m2c': [0.0038746800273656845,
  -0.0012637500185519457,
  0.7297147512435913],
 'obj_bb': [433, 315, 124, 122],
 'obj_id': 0,
 'cam_K': [2955.5368791923634,
  0.0,
  480.0,
  0.0,
  -2893.5525516872185,
  371.0,
  0.0,
  0.0,
  1.0]}

In [14]:
print(yaml.dump(gt_dict, default_flow_style=False))

Screenshot 2022-05-26 15_41_58_226306:
  cam_K:
  - 2955.5368791923634
  - 0.0
  - 480.0
  - 0.0
  - -2893.5525516872185
  - 371.0
  - 0.0
  - 0.0
  - 1.0
  cam_R_m2c:
  - 0.005844184663146734
  - 0.9999504089355469
  - -0.008065949194133282
  - 0.9998961687088013
  - -0.005949746817350388
  - -0.013125993311405182
  - -0.013173332437872887
  - -0.007988400757312775
  - -0.9998813271522522
  cam_t_m2c:
  - 0.0038746800273656845
  - -0.0012637500185519457
  - 0.7297147512435913
  obj_bb:
  - 433
  - 315
  - 124
  - 122
  obj_id: 0
Screenshot 2022-05-26 15_42_33_787889:
  cam_K:
  - 2993.4316165099845
  - 0.0
  - 480.0
  - 0.0
  - -2930.652550212997
  - 371.0
  - 0.0
  - 0.0
  - 1.0
  cam_R_m2c:
  - -0.43191200494766235
  - 0.7548061013221741
  - -0.49367982149124146
  - 0.6386807560920715
  - 0.6424485445022583
  - 0.42349350452423096
  - 0.6368193626403809
  - -0.13239187002182007
  - -0.7595613598823547
  cam_t_m2c:
  - -0.02852189913392067
  - 0.08361898362636566
  - 0.75668156147003

In [15]:
with open(os.path.join(output_path, "gt.yml"), "w") as f:
    yaml.dump(gt_dict, f)

In [16]:
with open(os.path.join(output_path, "info.yml"), "w") as f:
    yaml.dump(info_dict, f)